In [1]:
import pandas as pd
import numpy as np
import scipy.io.wavfile
import requests
import keyboard
import lxml.etree as etree
from multiprocessing import Process, set_start_method
import time
import re

In [2]:
session = requests.Session()

In [5]:
resp = session.get("http://localhost:5000/current")
root = etree.fromstring(resp.content)

In [68]:
import re

mtconnect_str = re.split('{|}' ,root.tag)[1]
mtconnect_str

'urn:mtconnect.org:MTConnectStreams:1.5'

In [ ]:
# get text inside paranthesis
mtconnect_str = root.tag.split('}')[0] + '}'

In [7]:
devices = {}

for device, sample in zip(root.iter(mtconnect_str + 'DeviceStream'), root.iter(mtconnect_str + 'DisplacementTimeSeries')):
    devices[device.attrib['name']] = np.array([float(x) for x in sample.text.split(' ')])

In [8]:
devices

{'sound0': array([-2399., -2474.,  -616., ...,  5241.,  3985.,  2722.]),
 'sound1': array([ 6542.,  6679.,  6792., ..., 10670., 10264.,  9835.]),
 'sound2': array([9668., 9233., 8790., ..., 7679., 7875., 8002.])}

In [35]:
root[1].tag

'{urn:mtconnect.org:MTConnectStreams:1.5}Streams'

In [49]:
sensor = 'sensor0'
# root.xpath("//x:DisplacementTimeSeries[@dataItemId=\"sensor0\"]")
root.xpath("//x:DisplacementTimeSeries[@dataItemId='" + sensor + "']", namespaces={'x': 'urn:mtconnect.org:MTConnectStreams:1.5'})[0].text

'-2399 -2474 -616 1994 4240 4314 2640 1181 -637 -3287 -4731 -3479 -1386 -833 -2653 -4561 -3508 -198 2083 1701 567 521 2321 4793 5048 2040 -2359 -3942 -1642 827 1184 475 1541 4620 6478 6174 5522 4831 4305 4181 3776 2720 1555 1116 1282 1646 1312 571 452 -27 -352 797 3292 6170 7359 5462 2329 787 1338 2308 2502 2234 2459 2176 259 -2737 -5223 -7582 -9959 -10948 -8800 -4357 -1462 -667 723 3417 3657 -440 -3532 -2251 890 2949 3196 2428 1532 1184 2336 4381 5724 4865 3691 3906 3795 1269 -2093 -3003 -2612 -1264 1856 4374 4128 2077 445 -591 -1068 -631 164 920 2035 3100 2200 -258 -2281 -2445 -1731 -2749 -4655 -5694 -6644 -8057 -8761 -7260 -5018 -2468 549 2809 3875 3924 2633 1180 1353 942 -2334 -6005 -6475 -4229 -1182 1648 1976 230 -115 2832 7476 11070 11964 10769 7876 2764 -2428 -4893 -3931 -1531 1018 3452 4029 3709 2627 158 -2568 -4029 -3334 -1313 536 1648 1979 1989 944 -501 365 3588 5064 2992 754 293 1328 3152 4538 4266 2100 -1792 -5006 -5070 -2284 1857 5676 8070 7892 4459 -402 -4224 -5972 -4514 

In [61]:
output = []
output.extend([1,2,3])
output.extend([4,5,6])
output

[1, 2, 3, 4, 5, 6]

In [71]:
output = []

def add_to_output(output, data):
    output.extend(data)
    return

add_to_output(output, [1,2,3])
output

[1, 2, 3]

In [70]:
root.xpath("//x:Header", namespaces={'x': mtconnect_str})[0].attrib

{'creationTime': '2023-10-25T17:12:30Z', 'sender': 'AcerSwiftX', 'instanceId': '1698253818', 'version': '1.7.0.2', 'bufferSize': '8192', 'nextSequence': '8454', 'firstSequence': '262', 'lastSequence': '8453'}

In [40]:
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def record(output: list, content, sensor):
    info('function record')
    root = etree.fromstring(content)
    device_root = root.xpath("//x:DisplacementTimeSeries[@dataItemId='" + sensor + "']", namespaces={'x': 'urn:mtconnect.org:MTConnectStreams:1.5'})
    for data in device_root:
        output.extend([np.int16(x) for x in data.text.split(' ')])

In [7]:
AGENT = 'http://localhost:5000/'
CURRENT = 'current'
PROBE = 'probe'
SAMP_RATE = 48000

session = requests.Session()

# Define a flag to control the loop
running = True

# Function to handle the key press event
def on_key_event(e):
    global running
    if e.event_type == keyboard.KEY_DOWN and e.name == 'space':
        running = False

# Register the key event listener
keyboard.on_press(on_key_event)

# Get first, last, time from current
resp = session.get(AGENT + CURRENT)
sound_root = etree.fromstring(resp.content)

NAMESPACE = re.split('{|}', sound_root.tag)[1]

header = sound_root.xpath("//x:Header", namespaces={'x': NAMESPACE})[0].attrib
start_time = pd.to_datetime(header['creationTime'])
f_seq = int(header['firstSequence'])
l_seq = int(header['lastSequence'])

devices = {device.attrib['uuid'] : [] for device in sound_root.xpath("//x:DeviceStream", namespaces={'x': NAMESPACE})}

In [44]:
out = []
record(out, resp.content, 'sensor0')
scipy.io.wavfile.write("output.wav", SAMP_RATE, np.array(out, dtype=np.int16))

function record
module name: __main__
parent process: 18352
process id: 22004


In [45]:
import wave

# Create a WAV file and write the audio data to it
with wave.open("output2.wav", "w") as wav_file:
    wav_file.setnchannels(1)  # Set the number of channels (1 for mono, 2 for stereo)
    wav_file.setsampwidth(2)  # Set the sample width in bytes (2 for 16-bit audio)
    wav_file.setframerate(SAMP_RATE)
    wav_file.writeframes(b''.join([int(x).to_bytes(2, byteorder='little', signed=True) for x in out]))

In [38]:
procs = []

for device in devices.keys():
    print(devices[device], device.split('-')[-1], resp.content)
    proc = Process(target=record, args=(devices[device], resp.content, device.split('-')[-1]))
    procs.append(proc)
    proc.start()
    
print(procs)

for proc in procs:
    proc.join()

devices

[] sensor0 b'<?xml version="1.0" encoding="UTF-8"?>\n<MTConnectStreams xmlns:m="urn:mtconnect.org:MTConnectStreams:1.5" xmlns="urn:mtconnect.org:MTConnectStreams:1.5" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="urn:mtconnect.org:MTConnectStreams:1.5 http://schemas.mtconnect.org/schemas/MTConnectStreams_1.5.xsd">\n  <Header creationTime="2023-10-28T01:48:08Z" sender="AcerSwiftX" instanceId="1698457491" version="1.7.0.2" bufferSize="8192" nextSequence="12082" firstSequence="3890" lastSequence="12081"/>\n  <Streams>\n    <DeviceStream name="sound0" uuid="imi-imt-vm20i-sensor0">\n      <ComponentStream component="Device" name="sound0" componentId="sound0">\n        <Samples>\n          <DisplacementTimeSeries dataItemId="sensor0" name="sensor0" sampleCount="2048" sampleRate="48000" sequence="12080" timestamp="2023-10-28T01:48:08.637508Z">-1210 302 3053 5300 4598 870 -2730 -3434 -2178 -1576 -2732 -4485 -4591 -2037 1608 4170 5154 4435 1721 -727 -1499 -2113 -2948

{'imi-imt-vm20i-sensor0': [],
 'imi-imt-vm20i-sensor1': [],
 'imi-imt-vm20i-sensor2': []}

In [ ]:
# Your main loop
print("Logging, press space to stop", start_time)
while running:
    SAMPLE = 'sample?from={}&to={}'.format(f_seq, l_seq)
    
    # request sample
    resp = session.get(AGENT + SAMPLE)
    sound_root = etree.fromstring(resp.content)
    
    # collect devices and data
    procs = []
    for device in devices.keys():
        proc = Process(target=record, args=(devices[device], resp.content, device.split('-')[-1]))
        procs.append(proc)
        proc.start()
    
    for proc in procs:
        proc.join()
        
    # update first and last sequence
    try:
        header = sound_root.xpath("//x:Header", namespaces={'x': NAMESPACE})[0].attrib
    except:
        print(f_seq, l_seq, SAMPLE)
        exit(1)
    f_seq = int(header['nextSequence'])
    l_seq = int(header['lastSequence'])
    
    time.sleep(1)
else:
    # collect the final sample
    SAMPLE = 'sample?from={}&to={}'.format(f_seq, l_seq)
    
    # request sample
    resp = session.get(AGENT + SAMPLE)
    sound_root = etree.fromstring(resp.content)
    
    # collect devices and data
    procs = []
    for device in devices.keys():
        proc = Process(target=record, args=(devices[device], resp.content, device.split('-')[-1]))
        procs.append(proc)
        proc.start()
        
    for proc in procs:
        proc.join()
        
# Unregister the key event listener when the loop exits
keyboard.unhook_all()

# Additional code after the loop
devices

In [46]:
sound_root.xpath("//x:DisplacementTimeSeries[@timestamp]")

<Element {urn:mtconnect.org:MTConnectStreams:1.5}MTConnectStreams at 0x2116becd280>

In [3]:
atime = pd.to_datetime(sound_root.xpath("//x:DisplacementTimeSeries[@sequence=12080]", namespaces={'x': NAMESPACE})[0].attrib['timestamp'])

NameError: name 'sound_root' is not defined

In [2]:
import pandas as pd
import numpy as np

atime = pd.to_datetime('2023-10-28 03:44:12+00:00')

In [14]:
print(atime.to_pydatetime(), sep='_')

2023-10-28 03:44:12+00:00


In [4]:
import datetime

t = datetime.datetime.utcnow()
pd.to_datetime(t)

Timestamp('2023-11-07 02:55:35.699754')

In [5]:
print(str(pd.to_datetime(t)))

2023-11-07 02:55:35.699754
